#**<font size="10" color='cornflowerblue'><ins>Welcome to exercise 10**
## **Note**: After choosing or changing an answer, run its block to save changes.
## **Note**: It is best to avoid Runtime -> Run all.



Block 1#


In [ ]:
#@title #**What is your ID and group number? (<font color="red">*Mandatory</font>)**


def initialize_colab():
  # Verify credentials
  if id_number < 1000 or group_number == 0 or group_number > 1000:
    raise Exception("Are you sure these details are correct?") 
  # Messages to user
  print("ID number saved: ", id_number)
  print("Group number saved: ", group_number)
  print("\nYou may now begin, good luck!")


try:
  # Imports and library initialization
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import numpy as np
  import matplotlib.pyplot as plt
  import seaborn as sns
  import pandas as pd
  from math import sqrt
  from scipy import stats
  import statsmodels.api as sm
  from ipywidgets import interact
  from tabulate import tabulate
  sns.set(style = "white", palette = "muted", color_codes = True)
  sns.set(rc = {'figure.figsize': (10, 10)});
  sns.set_context("talk")

  # Setting up global colab variables
  exercise_number = 10
  executed_blocks = {0, 1}
  block_mapping = {3: '1.1', 4: '1.2', 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 
                   11: '7.1', 12: '7.2', 14: '8.1', 15: '8.2'}
  answers = {}
  data = {}
  id_number =  0#@param {type:"integer"}
  group_number = 0 #@param {type:"integer"}
  initialize_colab()
  initialized = True
  
  # Setting up specific variables for the exercise
  degs_freedom = [2,5,10,20]
  np.random.seed(group_number)
  degs_freedom = np.random.choice(degs_freedom, size = len(degs_freedom), 
                                  replace = False)
  data['degs_freedom'] = list(degs_freedom)
  
except Exception as e:
  print(str(e))


# Dependency check function
def verify_dependencies(block_id, required_blocks_indices=[]):
  for block_index in required_blocks_indices:
    if block_index not in executed_blocks:
      raise Exception("You must first run block "+str(block_index)+"#")


# **********************  Statistics helper functions  **********************
def t_fun(deg_freedom,deg_freedom_extra,t_score):
  pval = (stats.t.sf((t_score), deg_freedom))  # two-sided pvalue = Prob(abs(t)>tt)
  if (t_score<0):
    pval=1-pval #
  print ("t(",deg_freedom ,") = ", t_score,", p-value:", pval)
  ttt = stats.t.rvs(deg_freedom,size=5000)
  plot_dist(ttt, height=5,bins=20,color="orange", fit="t")
  plt.axvline(t_score,color="black",linestyle="--",label="Chosen value")
  plt.title("t-distribution")


def F_fun(df1,df2,F_score):
  df1 = float(df1) ##k-1
  df2 =float(df2) ##N-k
  F_score = float(F_score)  ##MSB/MSW
  pval = (stats.f.sf(F_score, df1,df2))  # two-sided pvalue = Prob(abs(t)>tt)
  print("F(",df1,",",df2 ,") = ", F_score,", p-value:", round(pval,3),"\n")
  fff = stats.f.rvs(df1,df2,size=1000)
  sns.distplot(fff,hist=1,kde=0,axlabel="F-score",fit=stats.f,fit_kws={"color":"purple"})
  plt.xlim(0,max(10,F_score*1.1))
  plt.axvline(F_score,color="black",linestyle="--")
  plt.title("F-distribution")


def plot_dist(dist, bins="auto", kde=False, fit="None", color="b", stat="density",height=8, aspect=1):
  plt.figure(0);
  sns.displot(dist, kde=kde,bins=bins,stat=stat,color=color,height=height, aspect=aspect);
  if (fit=="Normal") & (stat=="density"):  
    d = stats.norm.fit(dist)
    mu = d[0]; sigma = d[1]
    x = np.linspace(mu - 3.5*sigma, mu + 3.5*sigma, 200) #limits of dist
    plt.plot(x, stats.norm.pdf(x, mu, sigma),color="black");
  elif (fit=="F") & (stat=="density"):  
    d = stats.f.fit(dist)
    dfn = d[0]; dfd = d[1]
    interval = stats.f.interval(.995,dfn,dfd) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.f.pdf(x, dfn, dfd),color="black");
  elif (fit=="gamma") & (stat=="density"):  
    d = stats.gamma.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.gamma.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.gamma.pdf(x, a, loc, scale),color="black");
  elif (fit=="t") & (stat=="density"):  
    d = stats.t.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.t.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.t.pdf(x, a, loc, scale),color="black");


def perform_fdr_t(t_vals, deg_freedom, alpha):
  old_t=t_vals.copy()
  t_vals=np.sort(t_vals)[::-1]
  pvals = [(stats.t.sf(np.abs(t_score), deg_freedom)) for t_score in t_vals]
  sig=0
  for i in range(len(pvals)):
    if pvals[i]<(alpha*(i+1)/len(t_vals)):
      sig=pvals[i]
  return (sig)


def boot_dist(dist, B=10000,replacement = True):
  sample_matrix = np.random.choice(dist,size=(len(dist),B),replace=replacement)
  means_dist = np.mean(sample_matrix,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist)


def boot_dist_F(data,IV,DV, B=10000, replacement = True):
  levels = np.unique(data[IV])
  F_vals=[]
  for i in range(B):
    samples = [np.random.choice(data[data[IV]==level][DV],size=len(data[data[IV]==level][DV]),
                                replace=replacement) for level in levels]
    f,_=stats.f_oneway(*samples) # average each column resulting in len(dist) X 1 vector
    F_vals.append(f) 
  return (F_vals)


def cohen_d(x,y):
  nx = len(x)
  ny = len(y)
  dof = nx + ny - 2
  return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


def plot_reg_results(x,y):
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
  term="+"*(intercept>0)
  if p_value<0.001: p_value=0.001
  pred_y = r"$\tildey$"
  reg_label = (f"{pred_y}={str(np.round(slope,1))}x{term}{str(np.round(intercept,1))}, p<{np.round(p_value,3)}")
  plt.figure(figsize=(7,7));
  sns.regplot(x=x,y=y);
  plt.title(reg_label);
  return ([slope,intercept,p_value])


def r_score(x, y):
    return round(stats.pearsonr(x, y)[0],3)


def plot_reg_and_minimize(betax,beta0,X,Y):
  student_x = X
  student_y = betax*student_x+beta0
  plt.figure(figsize=(12,8))
  sns.regplot( x=X,y=Y,fit_reg=False, color = "#154318")
  sns.lineplot(x=student_x, y=student_y, color = "r",label=("y="+str(betax)+"x+"+str(beta0)) )
  plt.xlabel('x', color='#1C2833')
  plt.ylabel('y', color='#1C2833')
  plt.ylim(min(Y)*.95,max(Y)*1.05)
  plt.xlim(min(X)*.95,max(X)*1.05)
  SSerror = np.round(np.sum((Y-student_y)**2))
  SST=np.round(np.sum((Y-Y.mean())**2))
  plt.legend(loc='upper left')
  plt.title(f"Your graph:\nSSE ={str(int(SSerror))}, SST={SST}")
  slope = (r_score(X,Y)*np.std(Y)/np.std(X))


def spearman_score(x, y):
    return round(stats.spearmanr(x, y)[0],3)


# **********************  Other helper functions  **********************
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '20px'
      rule.style.color = 'darkred'
      break
    }
  }
  '''))


# **********************  Helper class  **********************
class Polynomial(list): 
    def __repr__(self):
        # joiner[first, negative] = str
        joiner = {
            (True, True): '-',
            (True, False): '',
            (False, True): ' - ',
            (False, False): ' + '
        }
 
        result = []
        for power, coeff in reversed(list(enumerate(self))):
            j = joiner[not result, coeff < 0]
            coeff = abs(coeff)
            if coeff == 1 and power != 0:
                coeff = ''
 
            f = {0: '{}{}', 1: '{}{}x'}.get(power, '{}{}x^{}')
 
            result.append(f.format(j, coeff, power))
 
        return ''.join(result) or '0'

**<font size="6" color='cornflowerblue'><ins>Question 1</ins>** 

# Below you can see the distribution regarding the cannabis use between the ages 18-40.

# A researcher tested whether the distribution in her university is different. Therefore, she sampled 100 people between the ages 18-40. The results are as follows:

Block 2#


In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Cannabis use last year, between the ages 18-40</ins>** 

block_id = 2
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block2():
  np.random.seed(int(group_number))
  cannabis_dist = pd.DataFrame({"Only once":[13], "A few times":[7],"Regularly":[8],"Did not use":[74] })
  cannabis_dist = cannabis_dist+np.random.choice([-1,-2,0,1,2,3],4)
  cannabis_data = pd.DataFrame({"Only once":[9], "A few times":[11],"Regularly":[12],"Did not use":[68] })
  cannabis_data = cannabis_data+np.random.choice([-1,-2,0,1,2,3],4)
  #normalize to 100%:
  cannabis_dist = np.round(100*cannabis_dist/float(cannabis_dist.sum(axis=1)))
  cannabis_data = np.round(100*cannabis_data/float(cannabis_data.sum(axis=1)))
  cannabis_dist["Did not use"]+=100-cannabis_dist.sum(axis=1)
  cannabis_data["Did not use"]+=100-cannabis_data.sum(axis=1)
  print("Distribution of cannabis usage (general population):\n")
  print(tabulate(cannabis_dist, headers='keys', tablefmt='psql', showindex=False))
  print("\n\nResearcher's data on usage in her university:\n")
  print(tabulate(cannabis_data, headers='keys', tablefmt='psql', showindex=False))
  cva, vap = stats.chisquare(np.array(cannabis_data).T,f_exp=np.array(cannabis_dist).T)
  np.save('cva', cva)
  np.save('vap', vap)
  return cannabis_dist, cannabis_data


try:
  prepare_block()
  cannabis_dist, cannabis_data = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 3#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1.1</ins>** 
#@markdown #**The correct test is**

block_id = 3
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block3():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Chi Square for Independence", "Chi Square for Homogeneity"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 4#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1.2</ins>** 
#@markdown #**Calculate the Chi Square value and write it down (use the p-value calculator for p-value)**

block_id = 4
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block4():
  chi_square =  0#@param {type:"number"}
  #@markdown Is it significant at alpha < 0.05?
  chi_significant = False #@param {type:"boolean"}
  answers[block_id] = (chi_square, chi_significant)
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 5#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 2</ins>** 
#@markdown #A researcher was interested whether the influence of faculty (Engineering, Medicine and Law) on students' satisfaction (a score between o to 100) changes between genders. 
#@markdown #**What is the right test to use to answer his question?**

block_id = 5
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block5():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Two-Way ANOVA and testing the interaction between gender and faculty.", "Repeated Measures analysis and testing the difference between the faculties.", "Chi Square Test for independence between gender and faculty.", "Two-Way ANOVA and testing the main effects in the design."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 6#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 3</ins>** 
#@markdown #**What is the difference between Chi Square Test for Independence and Chi Square Test for Homogeneity?**

block_id = 6
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block6():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "It is the same test with a different name.", "In Chi Square Test for Independence we test the existence of a correlation between two categorical variables, while in Chi Square Test for Homogeneity we test the existence of a correlation between two ordinal variables.", "In Chi Square Test for Independence we test the existence of a dependence between two categorical variables, while in Chi Square Test for Homogeneity we test if a single categorical variable is distributed as expected (by a previous model, distribution, or sample)", "In Chi Square Test for Independence we ask whether two variables are different from each other, while in Chi Square Test for Homogeneity we ask whether two variables are the same."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 7#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4</ins>** 
#@markdown #**What is the central tendency measure relevant for Mann-Whitney Test?**

block_id = 7
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block7():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The mean of each sample, since we measure the difference between two groups.", "The median of each sample, since we measure the difference between the ranks of each group.", "The variance of each sample, since we use Mann-Whitney if each distribution is very skewed.", "The mode of each sample, since we use Mann-Whitney if there is more than one mode in every sample distribution."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 8#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5</ins>** 
#@markdown #A manager of two caffes, "Capulet Caffe" and "Montague Caffe", tested their quality of service. To do so, she counted the tips every waiter got in both caffes and compared the two distributions.
#@markdown #**What is correct?**

block_id = 8
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block8():
  Answer = "Choose the correct answer" #@param ["Choose the correct answer", "Both distributions were very different from each other, so the manager can't analyze the data at all.", "If we can assume that the distributions are normal, it's better to use Mann-Whitney Test.", "If we cannot assume that the distributions are normal, it's better to use T-Test.", "If we cannot assume that the distributions are normal, it's better to use Mann-Whitney Test."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 9#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6</ins>** 
#@markdown #A researcher tested whether cats and dogs differ in their desire for playing with a ball. Therefore, he recruited cats and dogs, let them play with a ball and measured the time they played. He used the Shift Function to test his hypothesis and got a significant effect for deciles 6 to 9 only (dogs - cats).
#@markdown #**What is possible?**

block_id = 9
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block9():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The variances of both groups were the same.", "The medians of both groups were the same.", "The difference between the 25 percentile and the 75 percentile is the same in both groups."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 10#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7</ins>** 
#@markdown #Let's get back to "Capulet Caffe" and "Montague Caffe". To test whether the two caffes differ in their service, the manager counted the tips every waiter got in both caffes and compared the two distributions. To do so, she sampled 10 waiters from every caffe. The researcher decided to perform the Mann-Whitney Test and she needs your help (alpha = 0.05).
#@markdown #**Running this block will present the sum of tips every waiter got in every caffe.**

block_id = 10
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block10():
  np.random.seed(group_number)
  capulet = np.round(np.random.gamma(12,10,10))+22
  np.random.seed(group_number+1)
  montague = np.round(np.random.gamma(12,10,10))
  ranks = stats.rankdata(np.concatenate([capulet, montague]))
  UMW,VP=stats.mannwhitneyu(capulet,montague,use_continuity=False)
  increase_font()
  print(f"Capulet:  {capulet}, median = {np.median(capulet)}\n")
  print(f"Montague: {montague}, median = {np.median(montague)}")
  np.save('ranks', ranks)
  data['UMW'], data['VP'] = UMW, VP
  return capulet, montague, ranks

try:
  prepare_block()
  capulet, montague, ranks = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 11#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.1</ins>** 
#@markdown #Give points to every score to get the statistic U value. You are asked to give points to every observation in each group, from 0 to 10. The number of points of every obsevation Capulet group will be the number of observations in Montague group that are lower than it. 
#@markdown - Write the answers with commas between them - the order is important. 
#@markdown - For example, if group A is 8,11,46 and group B is 7,16,41, then the correct answer for A is 1,1,3 
#@markdown - '0' means that there is no higher observation in the other group; '10' means that all rankings in the other group are higher than this ranking.

#@markdown #**Give points to each observation:**

block_id = 11
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block11():
  capulet_points = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  montague_points = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  capulet_points = [float(pts) for pts in capulet_points.split(",")] #eval(f"[{capulet_points}]")
  montague_points = [float(pts) for pts in montague_points.split(",")] #eval(f"[{montague_points}]")
  if len(capulet_points) != 10:
    raise Exception("You need to set 10 values seperated by a comma for capulet_points")
  if len(montague_points) != 10:
    raise Exception("You need to set 10 values seperated by a comma for montague_points")
  answers[block_id] = capulet_points, montague_points
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 12#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.2</ins>** 
#@markdown #Now sum the points of every group and write down its U. Then, pick the smallest U between the two groups, congratulations – it is your U value. 
#@markdown #**Run the block to view the P value, and decide whether you got a significant result.**
#@markdown Note: the p-value here is for a two-tail test

block_id = 12
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block12():
  U_statistic =  0 #@param {type:"number"}
  #@markdown Is it significant at alpha < 0.05?
  significant = False #@param {type:"boolean"}
  n1=float(len(capulet))
  n2=float(len(montague))
  T=stats.tiecorrect(ranks)
  meanrank = n1*n2/2.0
  sd = np.sqrt(T * n1 * n2 * (n1+n2+1) / 12.0)
  z = float((max(U_statistic,n1*n2-U_statistic) - meanrank) / sd)
  p = np.round(stats.norm.sf(abs(z)),3)
  print(f"p-value = {p}")
  answers[block_id] = U_statistic, significant
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

<font size="6" color='cornflowerblue'><ins>**Question 8**

Block 13#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Four Chi distributions</ins>** 
#@markdown #**Press the button below to see four different chi square distributions.**

block_id = 13
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block13():
  fig,ax = plt.subplots(ncols=len(degs_freedom),figsize = (28,6))
  colors = ["olive","darkred","blueviolet","deepskyblue"]
  for i in range(len(degs_freedom)):
        np.random.seed(int(group_number))
        curr_dist=np.random.chisquare(degs_freedom[i],10000)
        sns.kdeplot(curr_dist,ax=ax[i],fill=True,color=colors[i])
        ax[i].set_xlabel(f"Chi square")
        if i>0: ax[i].set_ylabel(f"")
        ax[i].set_xlim(0,40)
        ax[i].set_ylim(0,.36)
        ax[i].set_title(f"graph {i+1}")


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 14#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 8.1</ins>** 
#@markdown #**Match each distribution above to its degrees of freedom.**
#@markdown Hint: What is expected when you add more and more groups to a Chi squared test for homogenity? What would happen to the Chi Squared values?
block_id = 14
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block14():
  graph_1 = "Choose an answer" #@param ["Choose an answer",2,5,10,20]
  if graph_1 == "Choose an answer":
    raise Exception("How many degrees of freedom does graph 1 have?")
  graph_2 = "Choose an answer" #@param ["Choose an answer",2,5,10,20]
  if graph_2 == "Choose an answer":
    raise Exception("How many degrees of freedom does graph 2 have?")
  graph_3 = "Choose an answer" #@param ["Choose an answer",2,5,10,20]
  if graph_3 == "Choose an answer":
    raise Exception("How many degrees of freedom does graph 3 have?")
  graph_4 = "Choose an answer" #@param ["Choose an answer",2,5,10,20]
  if graph_4 == "Choose an answer":
    raise Exception("How many degrees of freedom does graph 4 have?")
  answers[block_id] = [int(graph_1),int(graph_2),int(graph_3),int(graph_4)]
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 15#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 8.2</ins>** 
#@markdown #Lets learn a few things about the chi square distribution. Complete the sentence below, based on the figures above:
#@markdown #**Under the null hypothesis, in Chi Square analysis, as the DF get ________, the chances to get a higher chi value are _________. The chances to reject the null hypothesis _________**

block_id = 15
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block15():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "bigger, smaller, will change.", "bigger, bigger, will not change.", "smaller, bigger, will get bigger.", "smaller, smaller, will get smaller."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


<br>
<br>
<hr>
<br>
<br>

**<p><font size="10"><font color='cornflowerblue'> Good job!</ins></p>** </font>

#**You finished answering the questions for this exercise! Here are some important things to consider:**


* You can view your answers by running the next block. 
*After changing an answer to a question, you will need to run the 'Show your answers' again for it to reflect changes.
* <b>If you change an answer to a question, you will need to run it again for changes to be saved. 


Block 16#

In [ ]:
#@title **<font color = cornflowerblue> <font size = 6>Show Your Answers <br>(Run to refresh after saving a question)**

block_id = 16


def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id))


def answers_printer():
  for block in sorted(answers):
    print("Question " + str(block_mapping[block]) + ": " + str(answers[block]))

try:
  prepare_block()
  answers_printer()
except Exception as e:
  print(str(e))


Block 17#

In [ ]:
#@title  **<p><font size="10"><font color='cornflowerblue'><ins> <---  💾 SAVE! </ins></p>**
#@markdown If you are ready to submit your answers, go ahead and press 'Save!'💾 . This will prompt you to download your answers file. After downloading you will need to submit the file to moodle. Good luck!

from google.colab import files
from shutil import make_archive
from os import path, remove
from zipfile import ZipFile

block_id = 17

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id - 1))


def answer_saver(file):
  for block in sorted(answers):
    file.write("Question " + str(block_mapping[block]) + ": " + str(answers[block])+'\n')


def submission_saver():
  # Writing answers
  f = open("ex"+str(exercise_number)+"_submission", "w")
  f.write("#"+str(id_number)+"\n")
  f.write("#"+str(group_number)+"\n")
  answer_saver(f) 
  f.write("#End of answers")
  f.close()

  # Writing data
  g = open("ex"+str(exercise_number)+"_data", "w")
  g.write(str(data))
  g.close()
  datazip = ZipFile("ex"+str(exercise_number)+"_data"+'.zip', 'w')
  datazip.write("ex"+str(exercise_number)+"_data")
  datazip.close()

  # Creating submission file
  zipObj = ZipFile("ex"+str(exercise_number)+'.zip', 'w')
  zipObj.write("ex"+str(exercise_number)+'_submission')
  zipObj.write("ex"+str(exercise_number)+"_data")
  zipObj.write("ranks.npy")
  zipObj.write("cva.npy")
  zipObj.write("vap.npy")
  zipObj.close()
  files.download("ex"+str(exercise_number)+'.zip') 

  # Message to user
  print("Well done! You are now downloading your submission file")


try:
  prepare_block()
  submission_saver()
except Exception as e:
  print(str(e))

